In [1]:
import pandas as pd
import numpy as np
import chardet
import re
import json
from json import loads, dumps
import os
import string
import neattext as nt
import neattext.functions as nfx 

In [2]:
df=pd.read_csv('cyclingDataComplete.csv')

In [3]:
df.dtypes

Unnamed: 0     int64
Question      object
Answers       object
dtype: object

In [4]:
df

,Unnamed: 0,Question,Answers
0,0,How do you take your bike with you if you're r...,"Every year, cycling circles have thousands of ..."
1,1,How do you take your bike with you if you're r...,"The quality of the products is very good, but ..."
2,2,How do you take your bike with you if you're r...,Id love to get these products out to more peop...
3,3,How do you take your bike with you if you're r...,How do I take my bike with me if Im riding acr...
4,4,How do you take your bike with you if you're r...,The next best option is to sling the bike fram...
...,...,...,...
177934,177934,"Seatpost 300mm diameter, not Selcof !","Check the yellow pages under ""telephone poles"""
177935,177935,"Seatpost 300mm diameter, not Selcof !","If you mean 31.6mm, check out Ritchey."
177936,177936,"Seatpost 300mm diameter, not Selcof !","Easton EC90 Zero, EC70 Zero, EC90 & EC70. Bont..."
177937,177937,What do you think the wholesale is on this bike:,A Scott Cr1 Pro 2005 with full Ultegra. Usuall...


In [5]:
df['Question']=df['Question'].astype('string')

In [6]:
df['Answers']=df['Answers'].astype('string')


In [ ]:
#noise scan
# df['Answers'].apply(lambda x:nt.TextFrame(x).noise_scan())

In [7]:
#remove names
df['CleanAnswer']=df['Answers'].apply(nfx.remove_dates)
df['CleanAnswer']=df['CleanAnswer'].apply(nfx.remove_emails)
df['CleanAnswer']=df['CleanAnswer'].apply(nfx.remove_non_ascii)
df['CleanAnswer']=df['CleanAnswer'].apply(nfx.remove_userhandles)
df['CleanAnswer']=df['CleanAnswer'].apply(nfx.remove_punctuations)
df['CleanAnswer']=df['CleanAnswer'].apply(lambda x:nfx.remove_custom_pattern(x,term_pattern=r'((1[0-2]|0?[1-9]):([0-5][0-9]) ?([AaPp][Mm]))'))
df['CleanAnswer']=df['CleanAnswer'].apply(lambda x:nfx.remove_custom_pattern(x,term_pattern=r'/^[0-9]{4}-(((0[13578]|(10|12))-(0[1-9]|[1-2][0-9]|3[0-1]))|(02-(0[1-9]|[1-2][0-9]))|((0[469]|11)-(0[1-9]|[1-2][0-9]|30)))$/'))

In [8]:
df['CleanAnswer']=df['CleanAnswer'].apply(lambda x:nfx.remove_custom_pattern(x,term_pattern=r'(\b\d{1,2}\D{0,3})?\b(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|(Nov|Dec)(?:ember)?)\D?(\d{1,2}(st|nd|rd|th)?)?(([,.\-\/])\D?)?((19[7-9]\d|20\d{2})|\d{2})*'))
df['CleanAnswer']=df['CleanAnswer'].apply(lambda x:nfx.remove_custom_pattern(x,term_pattern=r'(?<!\w)0\w+'))

In [10]:
df[100:600]

,Unnamed: 0,Question,Answers,CleanAnswer
100,100,Tyres stuck on rim - very odd!,The forum wont allow me to edit anymore. Just ...,The forum wont allow me to edit anymore Just l...
101,101,Need help,"I just got a new presta valve tube fitted , I ...",I just got a new presta valve tube fitted I b...
102,102,Need help,"Before you do any repairs to the tube, try to ...",Before you do any repairs to the tube try to f...
103,103,Need help,"here is the video of the new tube, theres a ho...",here is the video of the new tube theres a hol...
104,104,Need help,That looks like a manufacturing defect on the ...,That looks like a manufacturing defect on the ...
...,...,...,...,...
595,595,problem with blocked hole for front derailleur...,Im not quite sure what the significance of the...,Im not quite sure what the significance of the...
596,596,problem with blocked hole for front derailleur...,If you dont see the relevance of helping us un...,If you dont see the relevance of helping us un...
597,597,problem with blocked hole for front derailleur...,"bianchi, believe I know what youre talking abo...",bianchi believe I know what youre talking abou...
598,598,problem with blocked hole for front derailleur...,Can you push a [COLOR= #808080]14g-or-[/COLOR]...,Can you push a [COLOR= #808080]14gor[/COLOR]15...


In [10]:
df['CleanAnswer']=df['CleanAnswer'].apply(nfx.remove_special_characters)

In [11]:
df[400:700]

,Unnamed: 0,Question,Answers,CleanAnswer
400,400,Black bibs,"Not sure, Ive only tried Fox shorts so far. Th...",Not sure Ive only tried Fox shorts so far They...
401,401,Black bibs,The problem is they are wearing out and need n...,The problem is they are wearing out and need n...
402,402,Black bibs,That sucks. The stitches on my Fox shorts have...,That sucks The stitches on my Fox shorts have ...
403,403,Black bibs,Rode them today pretty good buy I did use a ne...,Rode them today pretty good buy I did use a ne...
404,404,Black bibs,Must be the saddle! Stiff is definitely better...,Must be the saddle Stiff is definitely better ...
...,...,...,...,...
595,595,problem with blocked hole for front derailleur...,Im not quite sure what the significance of the...,Im not quite sure what the significance of the...
596,596,problem with blocked hole for front derailleur...,If you dont see the relevance of helping us un...,If you dont see the relevance of helping us un...
597,597,problem with blocked hole for front derailleur...,"bianchi, believe I know what youre talking abo...",bianchi believe I know what youre talking abou...
598,598,problem with blocked hole for front derailleur...,Can you push a [COLOR= #808080]14g-or-[/COLOR]...,Can you push a COLOR 80808014gorCOLOR15g spoke...


In [12]:
df['CleanAnswer']=df['CleanAnswer'].apply(lambda x:nfx.remove_custom_pattern(x,term_pattern=r'(\b(?:On|[a-zA-Z])\b.*\b(?:email|protected|wrote|write|writes))'))
df['CleanAnswer']=df['CleanAnswer'].apply(lambda x:nfx.remove_custom_pattern(x,term_pattern=r'(\b(?:|[a-zA-Z])\b.*\b(?:email|protected|wrote|write|writes))'))


In [13]:
df[50:100]

,Unnamed: 0,Question,Answers,CleanAnswer
50,50,High school project please help,The fact that you have items related to real l...,The fact that you have items related to real l...
51,51,High school project please help,Подаръчни комплекти Bleude Chanel Versace Crys...,Bleude Chanel Versace Crystal Versace Eros T...
52,52,torque wrench choice,trying to decide between these 2. LifeLine Ess...,trying to decide between these 2 LifeLine Esse...
53,53,Rear Rim Brakes having a squeaking/compressing...,the rear rim brakes on my fx1 are sounding lik...,the rear rim brakes on my fx1 are sounding lik...
54,54,Rear Rim Brakes having a squeaking/compressing...,"Hi Hashir, It sounds like your rear rim brakes...",Hi Hashir It sounds like your rear rim brakes ...
55,55,Rear Rim Brakes having a squeaking/compressing...,ill align them tmrw as my tyre is skidding for...,ill align them tmrw as my tyre is skidding for...
56,56,Tour de France and tire diameters ?!,"Hi fellows, About twenty-years ago Tour de Fra...",Hi fellows About twentyyears ago Tour de Franc...
57,57,Tour de France and tire diameters ?!,great topic . Id ridden years (decades)ago on ...,great topic Id ridden years decadesago on 21 ...
58,58,Tour de France and tire diameters ?!,Actually GCN have videos to prove that wider t...,Actually GCN have videos to prove that wider t...
59,59,Tour de France and tire diameters ?!,"Hi fellows, Since we are on the topic of tires...",Hi fellows Since we are on the topic of tires ...


In [14]:
# df.to_csv('CyclingDataPrep1.csv')

In [15]:
delete_RowsB=df[df['CleanAnswer'].str.len() <= 15].index
df.drop(delete_RowsB , inplace=True)

In [16]:
df

,Unnamed: 0,Question,Answers,CleanAnswer
0,0,How do you take your bike with you if you're r...,"Every year, cycling circles have thousands of ...",Every year cycling circles have thousands of b...
1,1,How do you take your bike with you if you're r...,"The quality of the products is very good, but ...",The quality of the products is very good but t...
2,2,How do you take your bike with you if you're r...,Id love to get these products out to more peop...,Id love to get these products out to more peop...
3,3,How do you take your bike with you if you're r...,How do I take my bike with me if Im riding acr...,How do I take my bike with me if Im riding acr...
4,4,How do you take your bike with you if you're r...,The next best option is to sling the bike fram...,The next best option is to sling the bike fram...
...,...,...,...,...
177934,177934,"Seatpost 300mm diameter, not Selcof !","Check the yellow pages under ""telephone poles""",Check the yellow pages under telephone poles
177935,177935,"Seatpost 300mm diameter, not Selcof !","If you mean 31.6mm, check out Ritchey.",If you mean 316mm check out Ritchey
177936,177936,"Seatpost 300mm diameter, not Selcof !","Easton EC90 Zero, EC70 Zero, EC90 & EC70. Bont...",Easton EC90 Zero EC70 Zero EC90 EC70 Bontrage...
177937,177937,What do you think the wholesale is on this bike:,A Scott Cr1 Pro 2005 with full Ultegra. Usuall...,A Scott Cr1 Pro 2005 with full Ultegra Usually...


In [27]:
df=df.drop(df[df['CleanAnswer']==""].index, inplace = True)

In [21]:
# df.to_csv('CyclingDataPrep1.csv')

In [ ]:
result=df.to_json('Spokes.json',orient="records")
parsed = loads(result)
dumps(parsed, indent=4)